In [0]:
%tensorflow_version 1.x

In [0]:
!pip install object-detection-core
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

In [0]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from tqdm import tqdm
import re
from datetime import datetime
from multiprocessing import Process, Queue
from copy import deepcopy
from keras import applications, optimizers, backend as K
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Reshape, Conv2D, Activation
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, Callback, LambdaCallback
from keras.utils import np_utils
from keras.layers.noise import GaussianNoise
from keras.regularizers import l1
import pandas as pd
import numpy as np
from numpy.random import randint
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import io
import os
import sys
import csv, json, pickle
from lxml import etree

In [0]:
os.chdir('/content/gdrive/My Drive/')
from SecondStage_colab import second_stage_utils
from SecondStage_colab import file_utils

sys.path.append('/content/gdrive/My Drive/SecondStage_colab/')
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/SecondStage_colab/'

In [0]:
LOG_PATH = '/content/gdrive/My Drive/SecondStage_colab/output/'
LABEL_MAP_PATH = '/content/gdrive/My Drive/SecondStage_colab/label_map.pbtxt'
GPU = True

set paths 

In [0]:
TRAIN_RECORD = '/content/gdrive/My Drive/output/SecondStage_X/darkred_bluegreen_white_yellow/training_rot9_8-24_10_darkred_bluegreen_white_yellow.record'
EVAL_RECORD = '/content/gdrive/My Drive/output/SecondStage_X/darkred_bluegreen_white_yellow/test_rot3_8-24_10_darkred_bluegreen_white_yellow.record'
OUT_NAME = 'darkred_bluegreen_white_yellow'

bin & reg - rotation, cat - colours

In [0]:
TYPE = '_bin'    
#TYPE = '_cat'
#TYPE = '_reg'

In [0]:
from file_utils import save_json
from second_stage_utils import *

def train(train_record, conf, type, rep=1):

    out = str(type)

    timestamp = "{:%Y-%m-%d-%H-%M}".format(datetime.now())
    log_path = LOG_PATH + conf['name'] + '_' + out + '/' + timestamp + '-r' + str(rep) + '/'
    os.makedirs(log_path, exist_ok=True)
    save_json(log_path + '/experiment_config.json', conf)

    label_map = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)
    num_classes = label_map_util.get_max_label_map_index(label_map_util.load_labelmap(LABEL_MAP_PATH)) + 1

    X,Y,Z,_ = tf_record_load_crops([train_record])
    X_train, Y_train, Z_train = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_train = angle_to_bin(Z_train)

    eval_records = EVAL_RECORD

    X,Y,Z,_ = tf_record_extract_crops([eval_records], 1, 0.0, 0.0)
    X_val, Y_val, Z_val = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_val = angle_to_bin(Z_val)

    assert len(X_val) > 0 and len(Y_val) > 0 and len(Z_val) > 0, '{} is incomplete'.format(eval_records)

    outputs = None
    model_final = None
    summary = TensorBoardCustom(log_dir=log_path, label_map=label_map, AddCustomMetrics=(out == '_cat'))
    earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

    filepath=log_path+"model-{epoch:02d}.h5"
    checkpoint = ModelCheckpoint(filepath, verbose=1, period=20)
    if conf['optimizer'] == 'rmsprop':
        optimizer = optimizers.RMSprop(lr=conf['learning_rate'])
    elif conf['optimizer'] == 'adam':
        optimizer = optimizers.Adam(lr=conf['learning_rate'])
    else:
        raise Error('Unknown optimizer: ' + conf['optimizer'])

    loss_weights = [1.0 if out == '_cat' else 0.0, 1.0 if out == '_reg' else 0.0, 1.0 if out == '_bin' else 0.0]
    epochs = 0
    if out == '_cat': 
      epochs = conf['epochs_cat']
    elif out == '_reg': 
      epochs = conf['epochs_reg']
    else:
      epochs = conf['epochs_bin']

    mobilenet_base = applications.mobilenet.MobileNet(alpha = conf['alpha'],
                                                      weights = "imagenet",
                                                      include_top=False,
                                                      dropout = conf['dropout'],
                                                      input_shape = (
                                                      conf['img_size'],
                                                      conf['img_size'],
                                                        3
                                                      ))
    shape = (1, 1, int(1024 * conf['alpha']))
    x = GlobalAveragePooling2D()(mobilenet_base.output)
    x = Reshape(shape, name='reshape_1')(x)
    # Branch regression
    reg = Conv2D(1, (1, 1), padding='same', name='conv_reg')(x)
    reg = Activation('linear', name='act_linear')(reg)
    reg = Reshape((1,), name='reg_out')(reg)
    # Branch orientation classification with bins
    bin = Conv2D(NUM_ORI_BINS, (1, 1), padding='same', name='conv_bin')(x)
    bin = Activation('softmax', name='act_bin')(bin)
    bin = Reshape((NUM_ORI_BINS,), name='bin_out')(bin)
    # Branch classification
    x = GaussianNoise(stddev=0.2)(x, training = True) #0.1
    cat = Conv2D(num_classes, (1, 1), padding='same', name='conv_cat')(x) #activity_regularizer=l1(0.0001)
    cat = GaussianNoise(stddev=0.01)(cat, training = True)
    cat = Activation('softmax', name='act_softmax')(cat)
    cat = Reshape((num_classes,), name='cat_out')(cat)

    model_final = Model(inputs = mobilenet_base.input, outputs = [cat,reg,bin])
    model_final.compile(optimizer = optimizer,
                  loss={'cat_out': 'categorical_crossentropy',
                        'reg_out': angle_mse,
                        'bin_out': 'categorical_crossentropy',
                  },
                  loss_weights={'cat_out': loss_weights[0],
                                'reg_out': loss_weights[1],
                                'bin_out': loss_weights[2]},
                  metrics ={'cat_out': 'accuracy',
                            'reg_out': angle_mae,
                            'bin_out': angle_bin_rmse})
    model_final.fit(
        X_train,
        {'cat_out': Y_train, 'reg_out': Z_train, 'bin_out': Z2_train},
        validation_data=(X_val, [Y_val, Z_val, Z2_val]),
        batch_size=conf['batch_size'], epochs=epochs, verbose=1,
        callbacks=[summary, checkpoint, earlyStop],
        shuffle=True
    )

    model_final.save(log_path+"model-final.h5")
    print(log_path)
    print("Finished training for {}".format(conf['name']))

In [0]:
default_sstage_conf = {
    'dataset': 'default',
    'epochs_cat': 100,
    'epochs_reg': 60,
    'epochs_bin': 70,
    'optimizer': 'adam',
    'learning_rate': 1e-4,
    'dropout': 0.001,
    'alpha': 0.5,
    'img_size': 35,
    'repetions': 1,
    'batch_size': 4096,
}

def create_all_sstage_experiments():
    configs = []
    #configs.extend(create_sstage_default())
    configs.extend(create_sstage_dropouts_alphas())
    return configs

def create_sstage_dropouts_alphas():
    config = []
    dropouts = [0.001] #, 0.1, 0.2, 0.3
    alphas = [0.75]
    batches = [1024, 2048, 4096]
    for drop in dropouts:
      modified = deepcopy(default_sstage_conf)
      modified['name'] = "ssdef_drop" + str(drop)
      modified['dropout'] = drop
      for alp in alphas:
        modifiedB = deepcopy(modified)
        modifiedB['name'] = modifiedB['name'] + "_alpha" + str(alp)
        modifiedB['alpha'] = alp
        for batch in batches:
          modifiedC = deepcopy(modifiedB)
          modifiedC['name'] = modifiedC['name'] + "_batch_size" + str(batch)
          modifiedC['batch_size'] = batch
          config.append(deepcopy(modifiedC))
    return config

def create_sstage_default():
    config = []
    modified = deepcopy(default_sstage_conf)
    modified['name'] = "sstage_default"
    config.append(deepcopy(modified))
    return config


exp = create_all_sstage_experiments()
for config in tqdm(exp):
    print("or_conf", config)
    for r in range(config['repetions']):    
        conf = deepcopy(config)
        conf['name'] = conf['name']+'_'+TYPE
        print("Start training:  " + conf['name'])
              
        if not GPU:
            p = Process(
                target=train,
                args=(TRAIN_RECORD, conf, TYPE, r)
            )
            p.start() 
        else:
            train(TRAIN_RECORD, conf, TYPE, r)


  0%|          | 0/3 [00:00<?, ?it/s]

or_conf {'dataset': 'default', 'epochs_cat': 100, 'epochs_reg': 60, 'epochs_bin': 70, 'optimizer': 'adam', 'learning_rate': 0.0001, 'dropout': 0.001, 'alpha': 0.75, 'img_size': 35, 'repetions': 1, 'batch_size': 1024, 'name': 'ssdef_drop0.001_alpha0.75_batch_size1024'}
Start training:  ssdef_drop0.001_alpha0.75_batch_size1024__bin

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded 320760 crops from ['/content/gdrive/My Drive/output/SecondStage_X/darkred_bluegreen_white_yellow/training_rot9_8-24_10_darkred_bluegreen_white_yellow.record']








/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '







10633216/10626956 [==============================] - 2s 0us/step




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 320760 samples, validate on 58320 samples


Epoch 1/70
320760/320760 [==============================] - 38s 119us/step - loss: 5.6545 - cat_out_loss: 4.6475 - reg_out_loss: 10765.4289 - bin_out_loss: 5.6545 - cat_out_acc: 0.0273 - reg_out_angle_mae: 89.8333 - bin_out_angle_bin_rmse: 89.4529 - val_loss: 4.1517 - val_cat_out_loss: 5.2070 - val_reg_out_loss: 10740.8660 - val_bin_out_loss: 4.1517 - val_cat_out_acc: 0.0287 - val_reg_out_angle_mae: 89.5843 - val_bin_out_angle_bin_rmse: 43.8337

Epoch 2/70
320760/320760 [==============================] - 28s 86us/step - loss: 3.2610 - cat_out_loss: 5.2823 - reg_out_loss: 10775.1686 - bin_out_loss: 3.2610 - cat_out_acc: 0.0313 - reg_out_angle_mae: 89.7043 - bin_out_angle_bin_rmse: 25.5532 - val_loss: 2.9848 - val_cat_out_loss: 5.5976 - val_reg_out_loss: 10780.8587 - v


 33%|███▎      | 1/3 [07:39<15:19, 459.58s/it]

...

Train on 320760 samples, validate on 58320 samples
Epoch 1/70
320760/320760 [==============================] - 30s 95us/step - loss: 6.0832 - cat_out_loss: 4.8441 - reg_out_loss: 10805.2675 - bin_out_loss: 6.0832 - cat_out_acc: 0.0143 - reg_out_angle_mae: 90.0799 - bin_out_angle_bin_rmse: 98.5964 - val_loss: 5.6305 - val_cat_out_loss: 5.9448 - val_reg_out_loss: 10819.4039 - val_bin_out_loss: 5.6305 - val_cat_out_acc: 0.0119 - val_reg_out_angle_mae: 90.5919 - val_bin_out_angle_bin_rmse: 76.9000
Epoch 2/70
320760/320760 [==============================] - 25s 77us/step - loss: 4.1149 - cat_out_loss: 5.1477 - reg_out_loss: 10790.4799 - bin_out_loss: 4.1149 - cat_out_acc: 0.0137 - reg_out_angle_mae: 90.2984 - bin_out_angle_bin_rmse: 52.7091 - val_loss: 3.6089 - val_cat_out_loss: 6.0546 - val_reg_out_loss: 10772.2829 - val_bin_out_loss: 3.6089 - val_cat_out_acc: 0.0100 - val_reg_out_angle_mae: 90.3555 - val_bin_out_angle_bin_rmse: 12.7904
Epoch 3/70
320760/320760 [===========================


 67%|██████▋   | 2/3 [17:26<08:17, 497.88s/it]

or_conf {'dataset': 'default', 'epochs_cat': 100, 'epochs_reg': 60, 'epochs_bin': 70, 'optimizer': 'adam', 'learning_rate': 0.0001, 'dropout': 0.001, 'alpha': 0.75, 'img_size': 35, 'repetions': 1, 'batch_size': 4096, 'name': 'ssdef_drop0.001_alpha0.75_batch_size4096'}
Start training:  ssdef_drop0.001_alpha0.75_batch_size4096__bin
Loaded 320760 crops from ['/content/gdrive/My Drive/output/SecondStage_X/darkred_bluegreen_white_yellow/training_rot9_8-24_10_darkred_bluegreen_white_yellow.record']


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Train on 320760 samples, validate on 58320 samples
Epoch 1/70
320760/320760 [==============================] - 30s 93us/step - loss: 6.3951 - cat_out_loss: 4.8684 - reg_out_loss: 10790.9464 - bin_out_loss: 6.3951 - cat_out_acc: 0.0180 - reg_out_angle_mae: 89.9315 - bin_out_angle_bin_rmse: 102.0114 - val_loss: 6.1692 - val_cat_out_loss: 5.3175 - val_reg_out_loss: 10797.7152 - val_bin_out_loss: 6.1692 - val_cat_out_acc: 0.0178 - val_reg_out_angle_mae: 90.0033 - val_bin_out_angle_bin_rmse: 92.8584
Epoch 2/70
320760/320760 [==============================] - 25s 77us/step - loss: 5.3461 - cat_out_loss: 4.4741 - reg_out_loss: 10784.6636 - bin_out_loss: 5.3461 - cat_out_acc: 0.0165 - reg_out_angle_mae: 89.8635 - bin_out_angle_bin_rmse: 87.8421 - val_loss: 4.7944 - val_cat_out_loss: 5.6278 - val_reg_out_loss: 10760.9010 - val_bin_out_loss: 4.7944 - val_cat_out_acc: 0.0124 - val_reg_out_angle_mae: 89.6981 - val_bin_out_angle_bin_rmse: 55.9060
Epoch 3/70
320760/320760 [==========================

In [0]:
%load_ext tensorboard
%reload_ext tensorboard
# LOG_PATH did not work for some reason
%tensorboard --logdir '/content/gdrive/My Drive/SecondStage_colab/output/'

In [0]:
!pip install -U tensorboard >piplog 2>&1
!tensorboard dev upload --logdir '/content/gdrive/My Drive/SecondStage_colab/output/'

In [0]:
!pip install -U tensorboard >piplog 2>&1
!tensorboard dev list
# You must replace YOUR_EXPERIMENT_ID with the value output from the previous
# tensorboard `list` command or `upload` command.  For example
# `tensorboard dev delete --experiment_id pQpJNh00RG2Lf1zOe9BrQA`

## !tensorboard dev delete --experiment_id YOUR_EXPERIMENT_ID_HERE

In [0]:
!rm "/content/gdrive/My Drive/SecondStage_colab/output/" -rf
!ls "/content/gdrive/My Drive/SecondStage_colab/"